## 1. Load Datasets

In [1]:
## Utils
import re
import numpy as np
import pandas as pd
import itertools

## Classical Learner
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate

In [2]:
from sklearn.base import clone


class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:,1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[y-1][:,1] - clfs_predict[y][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[y-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [3]:
"""Dataset Parameter Setting"""
"""Load Dataset"""
cdcl2_b1_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_cdcl2_b1_bn_bl_corrected.csv")
cdcl2_b2_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_cdcl2_b2_bn_bl_corrected.csv")

pbcl2_b1_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_pbcl2_b1_bn_bl_corrected.csv")
pbcl2_b2_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_pbcl2_b2_bn_bl_corrected.csv")

pbno32_b1_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_pbno32_b1_bn_bl_corrected.csv")
pbno32_b2_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_pbno32_b2_bn_bl_corrected.csv")


"""Set Output Path"""
fileout = "C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/3results/HM_Classification/raw_data/baseline_ordinal_hm_model_output_b2_to_b1_bn_bl_corrected.csv"

In [4]:
cdcl2_b1_dset = pd.concat([pd.DataFrame(cdcl2_b1_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}), 
                           cdcl2_b1_dset.iloc[:, 5:]], axis=1)
cdcl2_b2_dset = pd.concat([pd.DataFrame(cdcl2_b2_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}), 
                           cdcl2_b2_dset.iloc[:, 5:]], axis=1)
cdcl2_all_dset = pd.concat([cdcl2_b1_dset, cdcl2_b2_dset], axis=0).reset_index(drop=True)

pbcl2_b1_dset = pd.concat([pd.DataFrame(pbcl2_b1_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}), 
                           pbcl2_b1_dset.iloc[:, 5:]], axis=1)
pbcl2_b2_dset = pd.concat([pd.DataFrame(pbcl2_b2_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}), 
                           pbcl2_b2_dset.iloc[:, 5:]], axis=1)
pbcl2_all_dset = pd.concat([pbcl2_b1_dset, pbcl2_b2_dset], axis=0).reset_index(drop=True)

pbno32_b1_dset = pd.concat([pd.DataFrame(pbno32_b1_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}), 
                            pbno32_b1_dset.iloc[:, 5:]], axis=1)
pbno32_b2_dset = pd.concat([pd.DataFrame(pbno32_b2_dset.iloc[:, 3]).rename(columns={'Concentration_uM': 'label'}),
                            pbno32_b2_dset.iloc[:, 5:]], axis=1)
pbno32_all_dset = pd.concat([pbno32_b1_dset, pbno32_b2_dset], axis=0).reset_index(drop=True)

In [5]:
hm_all_dset = pd.concat([cdcl2_all_dset, pbcl2_all_dset, pbno32_all_dset], axis=0).reset_index(drop=True)

In [6]:
## Get Label Encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(hm_all_dset.iloc[:,0])

LabelEncoder()

In [7]:
## CdCl2 Dataset
tmp1 = le.transform(cdcl2_b1_dset.iloc[:,0])
tmp2 = le.transform(cdcl2_b2_dset.iloc[:,0])

cdcl2_b1_dset = pd.concat([pd.DataFrame(tmp1).rename(columns={0:'label'}), cdcl2_b1_dset.iloc[:,1:]], axis=1)
cdcl2_b2_dset = pd.concat([pd.DataFrame(tmp2).rename(columns={0:'label'}), cdcl2_b2_dset.iloc[:,1:]], axis=1)

In [8]:
print(np.unique(cdcl2_b1_dset.label))
print(np.unique(cdcl2_b2_dset.label))

[0 2 3]
[0 1 2 3]


In [9]:
## Pbcl2 Dataset
tmp1 = le.transform(pbcl2_b1_dset.iloc[:,0])
tmp2 = le.transform(pbcl2_b2_dset.iloc[:,0])

pbcl2_b1_dset = pd.concat([pd.DataFrame(tmp1).rename(columns={0:'label'}), pbcl2_b1_dset.iloc[:,1:]], axis=1)
pbcl2_b2_dset = pd.concat([pd.DataFrame(tmp2).rename(columns={0:'label'}), pbcl2_b2_dset.iloc[:,1:]], axis=1)

In [10]:
print(np.unique(pbcl2_b1_dset.label))
print(np.unique(pbcl2_b2_dset.label))

[1 2 3]
[0 1 2 3]


In [11]:
## Pb(NO3)2 Dataset
tmp1 = le.transform(pbno32_b1_dset.iloc[:,0])
tmp2 = le.transform(pbno32_b2_dset.iloc[:,0])

pbno32_b1_dset = pd.concat([pd.DataFrame(tmp1).rename(columns={0:'label'}), pbno32_b1_dset.iloc[:,1:]], axis=1)
pbno32_b2_dset = pd.concat([pd.DataFrame(tmp2).rename(columns={0:'label'}), pbno32_b2_dset.iloc[:,1:]], axis=1)

In [12]:
print(np.unique(pbno32_b1_dset.label))
print(np.unique(pbno32_b2_dset.label))

[1 2 3]
[0 1 2 3]


In [13]:
## Define Train and Test Sets
cdcl2_train_set = cdcl2_b2_dset
tmp_dt = cdcl2_b2_dset[cdcl2_b2_dset.label==1]
cdcl2_test_set = pd.concat([cdcl2_b1_dset, tmp_dt.iloc[0:1,:]], axis=0).reset_index(drop=True)

pbcl2_train_set = pbcl2_b2_dset
tmp_dt = pbcl2_b2_dset[pbcl2_b2_dset.label==0]
pbcl2_test_set = pd.concat([pbcl2_b1_dset, tmp_dt.iloc[0:1,:]], axis=0).reset_index(drop=True)

pbno32_train_set = pbno32_b2_dset
tmp_dt = pbno32_b2_dset[pbno32_b2_dset.label==0]
pbno32_test_set = pd.concat([pbno32_b1_dset, tmp_dt.iloc[0:1,:]], axis=0).reset_index(drop=True)

In [14]:
cdcl2_train_set = cdcl2_train_set.iloc[:,:-1]
cdcl2_test_set = cdcl2_test_set.iloc[:,:-1]
pbcl2_train_set = pbcl2_train_set.iloc[:,:-1]
pbcl2_test_set = pbcl2_test_set.iloc[:,:-1]
pbno32_train_set = pbno32_train_set.iloc[:,:-1]
pbno32_test_set = pbno32_test_set.iloc[:,:-1]

In [15]:
X_cdcl2_dset = cdcl2_train_set.iloc[:, 1:].to_numpy(dtype='float32')
X_cdcl2_tset = cdcl2_test_set.iloc[:, 1:].to_numpy(dtype='float32')
y_cdcl2_dset = cdcl2_train_set.iloc[:,0].to_numpy(dtype='int64') 
y_cdcl2_tset = cdcl2_test_set.iloc[:,0].to_numpy(dtype='int64') 


X_pbcl2_dset = pbcl2_train_set.iloc[:, 1:].to_numpy(dtype='float32')
X_pbcl2_tset = pbcl2_test_set.iloc[:, 1:].to_numpy(dtype='float32')
y_pbcl2_dset = pbcl2_train_set.iloc[:,0].to_numpy(dtype='int64') 
y_pbcl2_tset = pbcl2_test_set.iloc[:,0].to_numpy(dtype='int64') 

X_pbno32_dset = pbno32_train_set.iloc[:, 1:].to_numpy(dtype='float32')
X_pbno32_tset = pbno32_test_set.iloc[:, 1:].to_numpy(dtype='float32')
y_pbno32_dset = pbno32_train_set.iloc[:,0].to_numpy(dtype='int64') 
y_pbno32_tset = pbno32_test_set.iloc[:,0].to_numpy(dtype='int64') 

## 2. Train Test Split

In [16]:
## 4.4 train and test split for Classical Learner
X_cdcl2_train, X_cdcl2_test, y_cdcl2_train, y_cdcl2_test = train_test_split(X_cdcl2_dset, y_cdcl2_dset, test_size = 0.2, 
                                                    random_state=123)

X_pbcl2_train, X_pbcl2_test, y_pbcl2_train, y_pbcl2_test = train_test_split(X_pbcl2_dset, y_pbcl2_dset, test_size = 0.2, 
                                                    random_state=123)

X_pbno32_train, X_pbno32_test, y_pbno32_train, y_pbno32_test = train_test_split(X_pbno32_dset, y_pbno32_dset, test_size = 0.2, 
                                                    random_state=123)

## 3. Classical Learner

### 3.1 Naive Bayes

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB

In [18]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(BernoulliNB())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [19]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[86 14  4  0]
 [17 72  0  0]
 [13 92  1  0]
 [ 0  8  7 86]]
Metrics
ACC: 0.612
BACC: 0.624
F1_micro: 0.612
F1_macro: 0.561
AUROC_OVR: 0.806
AUROC_OVO: 0.805
Precisio_micro: 0.612
Precisio_macro: 0.553
Recall_micro: 0.612
Recall_macro: 0.624


In [20]:
NB_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [21]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 18  98   6 378]
 [  0   1   0   0]
 [150   0   0 350]
 [500   0   0   0]]
Metrics
ACC: 0.013
BACC: 0.259
F1_micro: 0.013
F1_macro: 0.013
AUROC_OVR: 0.433
AUROC_OVO: 0.454
Precisio_micro: 0.013
Precisio_macro: 0.009
Recall_micro: 0.013
Recall_macro: 0.259


In [22]:
NB_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [23]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(BernoulliNB())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [24]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [ 19  78   1   3]
 [  0   2 104   0]
 [  0   0   0 104]]
Metrics
ACC: 0.938
BACC: 0.938
F1_micro: 0.938
F1_macro: 0.934
AUROC_OVR: 0.986
AUROC_OVO: 0.986
Precisio_micro: 0.938
Precisio_macro: 0.94
Recall_micro: 0.938
Recall_macro: 0.938


In [25]:
NB_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [26]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [395 100   3   2]
 [  4 439  57   0]
 [  0   0   0 500]]
Metrics
ACC: 0.438
BACC: 0.578
F1_micro: 0.438
F1_macro: 0.35
AUROC_OVR: 0.843
AUROC_OVO: 0.871
Precisio_micro: 0.438
Precisio_macro: 0.534
Recall_micro: 0.438
Recall_macro: 0.578


In [27]:
NB_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [28]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(BernoulliNB())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [29]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 81   8   0   0]
 [  2 101   0   1]
 [ 18   1  87   0]
 [  0   0   0 101]]
Metrics
ACC: 0.925
BACC: 0.926
F1_micro: 0.925
F1_macro: 0.923
AUROC_OVR: 0.975
AUROC_OVO: 0.975
Precisio_micro: 0.925
Precisio_macro: 0.928
Recall_micro: 0.925
Recall_macro: 0.926


In [30]:
NB_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [31]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [431  64   5   0]
 [  8 158 334   0]
 [  0   0 396 104]]
Metrics
ACC: 0.335
BACC: 0.501
F1_micro: 0.335
F1_macro: 0.267
AUROC_OVR: 0.628
AUROC_OVO: 0.662
Precisio_micro: 0.335
Precisio_macro: 0.436
Recall_micro: 0.335
Recall_macro: 0.501


In [32]:
NB_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [33]:
outF = open(fileout, "w")
outF.write("Naive_Bayes, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, NB_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, NB_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, NB_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, NB_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, NB_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, NB_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.2 Decision Tree

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [35]:
## Cdcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(DecisionTreeClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [36]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[104   0   0   0]
 [  2  87   0   0]
 [  0   2 102   2]
 [  0   1   0 100]]
Metrics
ACC: 0.982
BACC: 0.982
F1_micro: 0.982
F1_macro: 0.982
AUROC_OVR: 0.989
AUROC_OVO: 0.989
Precisio_micro: 0.982
Precisio_macro: 0.982
Recall_micro: 0.982
Recall_macro: 0.982


In [37]:
DT_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [38]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[140 171  32 157]
 [  0   1   0   0]
 [123  59  28 290]
 [181  17  74 228]]
Metrics
ACC: 0.264
BACC: 0.448
F1_micro: 0.264
F1_macro: 0.195
AUROC_OVR: 0.607
AUROC_OVO: 0.649
Precisio_micro: 0.264
Precisio_macro: 0.217
Recall_micro: 0.264
Recall_macro: 0.448


In [39]:
DT_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [40]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(DecisionTreeClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [41]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0  99   2   0]
 [  0   1 105   0]
 [  0   0   0 104]]
Metrics
ACC: 0.992
BACC: 0.993
F1_micro: 0.992
F1_macro: 0.993
AUROC_OVR: 0.995
AUROC_OVO: 0.995
Precisio_micro: 0.992
Precisio_macro: 0.993
Recall_micro: 0.992
Recall_macro: 0.993


In [42]:
DT_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [43]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [289 211   0   0]
 [181 186 133   0]
 [  0  21   3 476]]
Metrics
ACC: 0.547
BACC: 0.66
F1_micro: 0.547
F1_macro: 0.464
AUROC_OVR: 0.798
AUROC_OVO: 0.804
Precisio_micro: 0.547
Precisio_macro: 0.621
Recall_micro: 0.547
Recall_macro: 0.66


In [44]:
DT_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [45]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(DecisionTreeClassifier())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [46]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 88   1   0   0]
 [  0 103   1   0]
 [  1   1 104   0]
 [  0   0   0 101]]
Metrics
ACC: 0.99
BACC: 0.99
F1_micro: 0.99
F1_macro: 0.99
AUROC_OVR: 0.994
AUROC_OVO: 0.994
Precisio_micro: 0.99
Precisio_macro: 0.99
Recall_micro: 0.99
Recall_macro: 0.99


In [47]:
DT_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [48]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [303  62 135   0]
 [ 19 145 161 175]
 [  1   0 336 163]]
Metrics
ACC: 0.258
BACC: 0.443
F1_micro: 0.258
F1_macro: 0.214
AUROC_OVR: 0.505
AUROC_OVO: 0.543
Precisio_micro: 0.258
Precisio_macro: 0.26
Recall_micro: 0.258
Recall_macro: 0.443


In [49]:
DT_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [50]:
outF = open(fileout, "a")
outF.write("Decision_Tree, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, DT_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, DT_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, DT_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, DT_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, DT_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, DT_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.3 Logistic Regression

In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [52]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegression())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [53]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[104   0   0   0]
 [  0  89   0   0]
 [  0   0 106   0]
 [  0   0   0 101]]
Metrics
ACC: 1.0
BACC: 1.0
F1_micro: 1.0
F1_macro: 1.0
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 1.0
Precisio_macro: 1.0
Recall_micro: 1.0
Recall_macro: 1.0


In [54]:
LR_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [55]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 21 100   1 378]
 [  0   1   0   0]
 [  2 149  39 310]
 [158 342   0   0]]
Metrics
ACC: 0.041
BACC: 0.28
F1_micro: 0.041
F1_macro: 0.052
AUROC_OVR: 0.483
AUROC_OVO: 0.437
Precisio_micro: 0.041
Precisio_macro: 0.273
Recall_micro: 0.041
Recall_macro: 0.28


In [56]:
LR_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [57]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegression())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [58]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [59]:
LR_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [60]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [327 173   0   0]
 [  0   0 500   0]
 [  0   0   0 500]]
Metrics
ACC: 0.782
BACC: 0.836
F1_micro: 0.782
F1_macro: 0.63
AUROC_OVR: 0.996
AUROC_OVO: 0.998
Precisio_micro: 0.782
Precisio_macro: 0.751
Recall_micro: 0.782
Recall_macro: 0.836


In [61]:
LR_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [62]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegression())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [63]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 104   0   0]
 [  2   0 104   0]
 [  0   0   0 101]]
Metrics
ACC: 0.995
BACC: 0.995
F1_micro: 0.995
F1_macro: 0.995
AUROC_OVR: 0.999
AUROC_OVO: 0.999
Precisio_micro: 0.995
Precisio_macro: 0.995
Recall_micro: 0.995
Recall_macro: 0.995


In [64]:
LR_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [65]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [468  30   2   0]
 [  0 389 111   0]
 [  0   0 436  64]]
Metrics
ACC: 0.137
BACC: 0.353
F1_micro: 0.137
F1_macro: 0.127
AUROC_OVR: 0.545
AUROC_OVO: 0.609
Precisio_micro: 0.137
Precisio_macro: 0.319
Recall_micro: 0.137
Recall_macro: 0.353


In [66]:
LR_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [67]:
outF = open(fileout, "a")
outF.write("Logistic_Regression, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, LR_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, LR_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, LR_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, LR_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, LR_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, LR_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.3 Logistic Regression with CV

In [68]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

In [69]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegressionCV())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [70]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[104   0   0   0]
 [  0  89   0   0]
 [  0   0 106   0]
 [  0   0   0 101]]
Metrics
ACC: 1.0
BACC: 1.0
F1_micro: 1.0
F1_macro: 1.0
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 1.0
Precisio_macro: 1.0
Recall_micro: 1.0
Recall_macro: 1.0


In [71]:
LR_CV_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [72]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 21 100   0 379]
 [  0   1   0   0]
 [  2 149  40 309]
 [153 347   0   0]]
Metrics
ACC: 0.041
BACC: 0.28
F1_micro: 0.041
F1_macro: 0.053
AUROC_OVR: 0.512
AUROC_OVO: 0.448
Precisio_micro: 0.041
Precisio_macro: 0.28
Recall_micro: 0.041
Recall_macro: 0.28


In [73]:
LR_CV_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [74]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegressionCV())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [75]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 0.999
AUROC_OVO: 0.999
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [76]:
LR_CV_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [77]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [296 204   0   0]
 [  0   0 500   0]
 [  0   0  17 483]]
Metrics
ACC: 0.791
BACC: 0.843
F1_micro: 0.791
F1_macro: 0.638
AUROC_OVR: 0.999
AUROC_OVO: 1.0
Precisio_micro: 0.791
Precisio_macro: 0.743
Recall_micro: 0.791
Recall_macro: 0.843


In [78]:
LR_CV_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [79]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegressionCV())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [80]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 103   0   1]
 [  2   0 104   0]
 [  0   0   0 101]]
Metrics
ACC: 0.992
BACC: 0.993
F1_micro: 0.992
F1_macro: 0.992
AUROC_OVR: 0.999
AUROC_OVO: 0.999
Precisio_micro: 0.992
Precisio_macro: 0.992
Recall_micro: 0.992
Recall_macro: 0.993


In [81]:
LR_CV_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [82]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [433  58   9   0]
 [  0 384 116   0]
 [  0   0 440  60]]
Metrics
ACC: 0.157
BACC: 0.367
F1_micro: 0.157
F1_macro: 0.14
AUROC_OVR: 0.58
AUROC_OVO: 0.641
Precisio_micro: 0.157
Precisio_macro: 0.335
Recall_micro: 0.157
Recall_macro: 0.367


In [83]:
LR_CV_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [84]:
outF = open(fileout, "a")
outF.write("Logistic_Regression_CV, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, LR_CV_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, LR_CV_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, LR_CV_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, LR_CV_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, LR_CV_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, LR_CV_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.4 MLP

In [85]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [86]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(MLPClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [87]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[103   0   0   1]
 [  0  89   0   0]
 [  0   0 106   0]
 [  0   0   0 101]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [88]:
MLP_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [89]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 22  87   8 383]
 [  0   1   0   0]
 [  1  35 115 349]
 [  2 497   1   0]]
Metrics
ACC: 0.092
BACC: 0.318
F1_micro: 0.092
F1_macro: 0.114
AUROC_OVR: 0.444
AUROC_OVO: 0.416
Precisio_micro: 0.092
Precisio_macro: 0.452
Recall_micro: 0.092
Recall_macro: 0.318


In [90]:
MLP_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [91]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(MLPClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [92]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [93]:
MLP_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [94]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [268 232   0   0]
 [ 40  54 406   0]
 [  0   0   0 500]]
Metrics
ACC: 0.759
BACC: 0.819
F1_micro: 0.759
F1_macro: 0.623
AUROC_OVR: 0.977
AUROC_OVO: 0.984
Precisio_micro: 0.759
Precisio_macro: 0.704
Recall_micro: 0.759
Recall_macro: 0.819


In [95]:
MLP_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [96]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(MLPClassifier())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [97]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 104   0   0]
 [  1   0 105   0]
 [  0   0   0 101]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.997
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.997
Recall_micro: 0.998
Recall_macro: 0.998


In [98]:
MLP_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [99]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [318  12  70 100]
 [ 25 384  87   4]
 [  0  17  20 463]]
Metrics
ACC: 0.375
BACC: 0.531
F1_micro: 0.375
F1_macro: 0.289
AUROC_OVR: 0.651
AUROC_OVO: 0.691
Precisio_micro: 0.375
Precisio_macro: 0.335
Recall_micro: 0.375
Recall_macro: 0.531


In [100]:
MLP_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [101]:
outF = open(fileout, "a")
outF.write("MLP, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, MLP_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, MLP_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, MLP_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, MLP_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, MLP_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, MLP_pbno32_tset_res)))
outF.write('\n')
outF.close()

### Random Forest

In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [103]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(RandomForestClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [104]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[103   0   0   1]
 [  0  89   0   0]
 [  0   1 103   2]
 [  0   0   0 101]]
Metrics
ACC: 0.99
BACC: 0.991
F1_micro: 0.99
F1_macro: 0.99
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.99
Precisio_macro: 0.99
Recall_micro: 0.99
Recall_macro: 0.991


In [105]:
RF_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [106]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  3  37   1 459]
 [  0   1   0   0]
 [109  11   0 380]
 [  8   2   0 490]]
Metrics
ACC: 0.329
BACC: 0.496
F1_micro: 0.329
F1_macro: 0.146
AUROC_OVR: 0.64
AUROC_OVO: 0.692
Precisio_micro: 0.329
Precisio_macro: 0.103
Recall_micro: 0.329
Recall_macro: 0.496


In [107]:
RF_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [108]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(RandomForestClassifier())
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [109]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [110]:
RF_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [111]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [332 168   0   0]
 [ 46  18 436   0]
 [  0   0   1 499]]
Metrics
ACC: 0.736
BACC: 0.802
F1_micro: 0.736
F1_macro: 0.606
AUROC_OVR: 0.99
AUROC_OVO: 0.993
Precisio_micro: 0.736
Precisio_macro: 0.726
Recall_micro: 0.736
Recall_macro: 0.802


In [112]:
RF_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [113]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(RandomForestClassifier())
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [114]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 103   0   1]
 [  1   0 105   0]
 [  0   0   0 101]]
Metrics
ACC: 0.995
BACC: 0.995
F1_micro: 0.995
F1_macro: 0.995
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.995
Precisio_macro: 0.995
Recall_micro: 0.995
Recall_macro: 0.995


In [115]:
RF_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [116]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [380  39  49  32]
 [  2 111 387   0]
 [  0   0 267 233]]
Metrics
ACC: 0.44
BACC: 0.58
F1_micro: 0.44
F1_macro: 0.344
AUROC_OVR: 0.739
AUROC_OVO: 0.784
Precisio_micro: 0.44
Precisio_macro: 0.423
Recall_micro: 0.44
Recall_macro: 0.58


In [117]:
RF_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [118]:
outF = open(fileout, "a")
outF.write("Random_Forest, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, RF_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, RF_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, RF_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, RF_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, RF_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, RF_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.6 Linear SVM

In [119]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [120]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'linear', probability=True))
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [121]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[104   0   0   0]
 [  0  89   0   0]
 [  0   0 106   0]
 [  0   0   0 101]]
Metrics
ACC: 1.0
BACC: 1.0
F1_micro: 1.0
F1_macro: 1.0
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 1.0
Precisio_macro: 1.0
Recall_micro: 1.0
Recall_macro: 1.0


In [122]:
LinSVM_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [123]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 18 100   0 382]
 [  0   1   0   0]
 [  4 149  65 282]
 [129 371   0   0]]
Metrics
ACC: 0.056
BACC: 0.292
F1_micro: 0.056
F1_macro: 0.072
AUROC_OVR: 0.537
AUROC_OVO: 0.481
Precisio_micro: 0.056
Precisio_macro: 0.28
Recall_micro: 0.056
Recall_macro: 0.292


In [124]:
LinSVM_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [125]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'linear', probability=True))
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [126]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [127]:
LinSVM_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [128]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [301 199   0   0]
 [  0   0 500   0]
 [  0   0   0 500]]
Metrics
ACC: 0.799
BACC: 0.85
F1_micro: 0.799
F1_macro: 0.644
AUROC_OVR: 0.999
AUROC_OVO: 0.999
Precisio_micro: 0.799
Precisio_macro: 0.751
Recall_micro: 0.799
Recall_macro: 0.85


In [129]:
LinSVM_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [130]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'linear', probability=True))
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [131]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 104   0   0]
 [  1   0 105   0]
 [  0   0   0 101]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.997
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.997
Recall_micro: 0.998
Recall_macro: 0.998


In [132]:
LinSVM_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [133]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [388  97  15   0]
 [  0 373 127   0]
 [  0   0 455  45]]
Metrics
ACC: 0.18
BACC: 0.384
F1_micro: 0.18
F1_macro: 0.15
AUROC_OVR: 0.609
AUROC_OVO: 0.677
Precisio_micro: 0.18
Precisio_macro: 0.355
Recall_micro: 0.18
Recall_macro: 0.384


In [134]:
LinSVM_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [135]:
outF = open(fileout, "a")
outF.write("Linear_SVM, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, LinSVM_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, LinSVM_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, LinSVM_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, LinSVM_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, LinSVM_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, LinSVM_pbno32_tset_res)))
outF.write('\n')
outF.close()

### 3.7 RBF SVM ( Nonlinear SVM)

In [136]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [137]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'rbf', probability=True))
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_cdcl2_train)
X_test_sds = scaler.transform(X_cdcl2_test)
y_train = y_cdcl2_train
y_test = y_cdcl2_test
X_tset = X_cdcl2_tset
y_tset = y_cdcl2_tset
clf.fit(X_train_sds, y_train)

In [138]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[103   0   0   1]
 [  1  88   0   0]
 [  0   3 103   0]
 [  0   1   0 100]]
Metrics
ACC: 0.985
BACC: 0.985
F1_micro: 0.985
F1_macro: 0.985
AUROC_OVR: 0.999
AUROC_OVO: 0.999
Precisio_micro: 0.985
Precisio_macro: 0.984
Recall_micro: 0.985
Recall_macro: 0.985


In [139]:
RBFSVM_cdcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [140]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[ 19  96   1 384]
 [  0   1   0   0]
 [  1 150   0 349]
 [  9 491   0   0]]
Metrics
ACC: 0.013
BACC: 0.26
F1_micro: 0.013
F1_macro: 0.019
AUROC_OVR: 0.439
AUROC_OVO: 0.405
Precisio_micro: 0.013
Precisio_macro: 0.164
Recall_micro: 0.013
Recall_macro: 0.26


In [141]:
RBFSVM_cdcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [142]:
## Pbcl2 Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'rbf', probability=True))
scaler.fit(X_cdcl2_train)
X_train_sds = scaler.transform(X_pbcl2_train)
X_test_sds = scaler.transform(X_pbcl2_test)
y_train = y_pbcl2_train
y_test = y_pbcl2_test
X_tset = X_pbcl2_tset
y_tset = y_pbcl2_tset
clf.fit(X_train_sds, y_train)

In [143]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 100   1   0]
 [  0   0 106   0]
 [  0   0   0 104]]
Metrics
ACC: 0.998
BACC: 0.998
F1_micro: 0.998
F1_macro: 0.998
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.998
Precisio_macro: 0.998
Recall_micro: 0.998
Recall_macro: 0.998


In [144]:
RBFSVM_pbcl2_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [145]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [334 166   0   0]
 [  0   0 500   0]
 [  0   0   0 500]]
Metrics
ACC: 0.777
BACC: 0.833
F1_micro: 0.777
F1_macro: 0.626
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.777
Precisio_macro: 0.751
Recall_micro: 0.777
Recall_macro: 0.833


In [146]:
RBFSVM_pbcl2_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [147]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'rbf', probability=True))
scaler.fit(X_pbno32_train)
X_train_sds = scaler.transform(X_pbno32_train)
X_test_sds = scaler.transform(X_pbno32_test)
y_train = y_pbno32_train
y_test = y_pbno32_test
X_tset = X_pbno32_tset
y_tset = y_pbno32_tset
clf.fit(X_train_sds, y_train)

In [148]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[ 89   0   0   0]
 [  0 103   1   0]
 [  1   0 105   0]
 [  0   0   0 101]]
Metrics
ACC: 0.995
BACC: 0.995
F1_micro: 0.995
F1_macro: 0.995
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 0.995
Precisio_macro: 0.995
Recall_micro: 0.995
Recall_macro: 0.995


In [149]:
RBFSVM_pbno32_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [150]:
### Independent Test Result
X_tset_sds = scaler.transform(X_tset)
yp_test = clf.predict(X_tset_sds)
ys_test = clf.predict_proba(X_tset_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_tset, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_tset, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_tset, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_tset, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_tset, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_tset, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_tset, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_tset, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_tset, yp_test, average='macro'),3)))

Confusion Matrix
[[  1   0   0   0]
 [249  21 230   0]
 [  1 201 298   0]
 [  0   0 448  52]]
Metrics
ACC: 0.248
BACC: 0.436
F1_micro: 0.248
F1_macro: 0.165
AUROC_OVR: 0.709
AUROC_OVO: 0.786
Precisio_micro: 0.248
Precisio_macro: 0.351
Recall_micro: 0.248
Recall_macro: 0.436


In [151]:
RBFSVM_pbno32_tset_res = [round(accuracy_score(y_tset, yp_test), 3), round(balanced_accuracy_score(y_tset, yp_test), 3),
                 round(f1_score(y_tset, yp_test, average='micro'),3), round(f1_score(y_tset, yp_test, average='macro'),3),
                 round(roc_auc_score(y_tset, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_tset, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_tset, yp_test, average='micro'),3), round(precision_score(y_tset, yp_test, average='macro'),3),
                 round(recall_score(y_tset, yp_test, average='micro'),3), round(recall_score(y_tset, yp_test, average='macro'),3)]

In [152]:
outF = open(fileout, "a")
outF.write("RBF_SVM, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('Cdcl2 DevSet, ')
outF.write(', '.join(map(str, RBFSVM_cdcl2_devset_res)))
outF.write('\n')
outF.write('Cdcl2 IndSet, ')
outF.write(', '.join(map(str, RBFSVM_cdcl2_tset_res)))
outF.write('\n')
outF.write('Pbcl2 DevSet, ')
outF.write(', '.join(map(str, RBFSVM_pbcl2_devset_res)))
outF.write('\n')
outF.write('Pbcl2 IndSet, ')
outF.write(', '.join(map(str, RBFSVM_pbcl2_tset_res)))
outF.write('\n')
outF.write('Pb(No3)2 DevSet, ')
outF.write(', '.join(map(str, RBFSVM_pbno32_devset_res)))
outF.write('\n')
outF.write('Pb(No3)2 IndSet, ')
outF.write(', '.join(map(str, RBFSVM_pbno32_tset_res)))
outF.write('\n')
outF.close()